In [1]:
# import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA


### Step 1: Data collection and preprocessing

In [2]:
df = pd.read_excel('/Users/drlisamk/Documents/data_driven_airbnb/financial_data.xlsx', header=1)

In [3]:
df
type(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               268 non-null    object 
 1   Type               268 non-null    object 
 2   Confirmation Code  268 non-null    object 
 3   Start Date         268 non-null    object 
 4   Nights             268 non-null    int64  
 5   Guest              268 non-null    object 
 6   Listing            268 non-null    object 
 7   Details            6 non-null      object 
 8   Reference          0 non-null      float64
 9   Currency           268 non-null    object 
 10  Amount             268 non-null    float64
 11  Paid Out           0 non-null      float64
 12  Host Fee           262 non-null    float64
 13  Cleaning Fee       262 non-null    float64
 14  Gross Earnings     266 non-null    float64
 15  Earnings Year      0 non-null      float64
dtypes: float64(7), int64(1), o

In [4]:
# Handling Missing Values
df.isna()  # Check for missing values
df.dropna()  # Remove rows or columns with missing values
df.fillna(0)  # Fill missing values with a specific value
#df.drop(['Guest','Currency'], axis=1, inplace=True)

,Date,Type,Confirmation Code,Start Date,Nights,Guest,Listing,Details,Reference,Currency,Amount,Paid Out,Host Fee,Cleaning Fee,Gross Earnings,Earnings Year
0,07/08/2023,Reservation,HM2RWXRDER,07/07/2023,2,Michelle Wells,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",0,0.0,ZAR,1332.35,0.0,47.65,0.0,1380.0,0.0
1,07/07/2023,Reservation,HMTBW5FD2J,07/06/2023,2,Lila Bounekta,"Skye Beach House: seaviews, sundeck, surf, WiFi",0,0.0,ZAR,2297.85,0.0,82.15,180.0,2380.0,0.0
2,07/07/2023,Resolution Adjustment,HM8YAZX25F,06/25/2023,3,Shahieda,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",Resolution adjustment for resolution 30043164,0.0,ZAR,-1737.90,0.0,0.00,0.0,-1737.9,0.0
3,07/04/2023,Reservation,HMKD9245B4,07/03/2023,5,Katlego Maeko,Skye Crystal Sea Shack,0,0.0,ZAR,3137.88,0.0,112.12,0.0,3250.0,0.0
4,07/04/2023,Reservation,HMCSYKHR9E,07/03/2023,3,Khodani Rasikhinya,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",0,0.0,ZAR,1593.08,0.0,56.92,0.0,1650.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,01/04/2023,Reservation,HMF5ABRZAR,01/03/2023,1,Helena Landskron,Skye Ocean Nest: tiny beach home for 2 - fast ...,0,0.0,ZAR,1255.15,0.0,44.85,0.0,1300.0,0.0
264,01/03/2023,Reservation,HMP9SSCWRD,01/02/2023,2,Meleshni Govender,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",0,0.0,ZAR,4827.50,0.0,172.50,0.0,5000.0,0.0
265,01/03/2023,Reservation,HMJMJYSH2Y,01/02/2023,1,Kornelius Filbinger,Skye Ocean Nest: tiny beach home for 2 - fast ...,0,0.0,ZAR,772.38,0.0,27.62,0.0,800.0,0.0
266,01/02/2023,Reservation,HMJYK24N4S,01/01/2023,1,Louis Swart,Skye Ocean Nest: tiny beach home for 2 - fast ...,0,0.0,ZAR,772.40,0.0,27.60,0.0,800.0,0.0


In [5]:
# Handling Duplicates
df.duplicated()  # Check for duplicate rows
df.drop_duplicates()  # Remove duplicate rows


,Date,Type,Confirmation Code,Start Date,Nights,Guest,Listing,Details,Reference,Currency,Amount,Paid Out,Host Fee,Cleaning Fee,Gross Earnings,Earnings Year
0,07/08/2023,Reservation,HM2RWXRDER,07/07/2023,2,Michelle Wells,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",NaN,NaN,ZAR,1332.35,NaN,47.65,0.0,1380.0,NaN
1,07/07/2023,Reservation,HMTBW5FD2J,07/06/2023,2,Lila Bounekta,"Skye Beach House: seaviews, sundeck, surf, WiFi",NaN,NaN,ZAR,2297.85,NaN,82.15,180.0,2380.0,NaN
2,07/07/2023,Resolution Adjustment,HM8YAZX25F,06/25/2023,3,Shahieda,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",Resolution adjustment for resolution 30043164,NaN,ZAR,-1737.90,NaN,NaN,NaN,-1737.9,NaN
3,07/04/2023,Reservation,HMKD9245B4,07/03/2023,5,Katlego Maeko,Skye Crystal Sea Shack,NaN,NaN,ZAR,3137.88,NaN,112.12,0.0,3250.0,NaN
4,07/04/2023,Reservation,HMCSYKHR9E,07/03/2023,3,Khodani Rasikhinya,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",NaN,NaN,ZAR,1593.08,NaN,56.92,0.0,1650.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,01/04/2023,Reservation,HMF5ABRZAR,01/03/2023,1,Helena Landskron,Skye Ocean Nest: tiny beach home for 2 - fast ...,NaN,NaN,ZAR,1255.15,NaN,44.85,0.0,1300.0,NaN
264,01/03/2023,Reservation,HMP9SSCWRD,01/02/2023,2,Meleshni Govender,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",NaN,NaN,ZAR,4827.50,NaN,172.50,0.0,5000.0,NaN
265,01/03/2023,Reservation,HMJMJYSH2Y,01/02/2023,1,Kornelius Filbinger,Skye Ocean Nest: tiny beach home for 2 - fast ...,NaN,NaN,ZAR,772.38,NaN,27.62,0.0,800.0,NaN
266,01/02/2023,Reservation,HMJYK24N4S,01/01/2023,1,Louis Swart,Skye Ocean Nest: tiny beach home for 2 - fast ...,NaN,NaN,ZAR,772.40,NaN,27.60,0.0,800.0,NaN


In [49]:
#NOT NEEDED HERE
#Data Type Conversion
#df.astype(dtype)  # Convert columns to a specific data type
#pd.to_datetime(df['date_column'])  # Convert a column to datetime type

In [51]:
# Data Transformation
#df.rename(columns={'old_name': 'new_name'})  # Rename columns
#df[df['column'] > 10]  # Filter rows based on a condition
#df['new_column'] = df['column1'] + df['column2']  # Add or modify columns

,Date,Type,Confirmation Code,Start Date,Nights,Guest,Listing,Currency,Amount,Host Fee,Cleaning Fee,Gross Earnings
0,07/08/2023,Reservation,HM2RWXRDER,07/07/2023,2,Michelle Wells,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",ZAR,1332.35,47.65,0.0,1380.0
1,07/07/2023,Reservation,HMTBW5FD2J,07/06/2023,2,Lila Bounekta,"Skye Beach House: seaviews, sundeck, surf, WiFi",ZAR,2297.85,82.15,180.0,2380.0
2,07/07/2023,Resolution Adjustment,HM8YAZX25F,06/25/2023,3,Shahieda,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",ZAR,-1737.90,0.00,0.0,-1737.9
3,07/04/2023,Reservation,HMKD9245B4,07/03/2023,5,Katlego Maeko,Skye Crystal Sea Shack,ZAR,3137.88,112.12,0.0,3250.0
4,07/04/2023,Reservation,HMCSYKHR9E,07/03/2023,3,Khodani Rasikhinya,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",ZAR,1593.08,56.92,0.0,1650.0
...,...,...,...,...,...,...,...,...,...,...,...,...
263,01/04/2023,Reservation,HMF5ABRZAR,01/03/2023,1,Helena Landskron,Skye Ocean Nest: tiny beach home for 2 - fast ...,ZAR,1255.15,44.85,0.0,1300.0
264,01/03/2023,Reservation,HMP9SSCWRD,01/02/2023,2,Meleshni Govender,"Skye Sea Cottage: seaviews, deck, fireplace, wifi",ZAR,4827.50,172.50,0.0,5000.0
265,01/03/2023,Reservation,HMJMJYSH2Y,01/02/2023,1,Kornelius Filbinger,Skye Ocean Nest: tiny beach home for 2 - fast ...,ZAR,772.38,27.62,0.0,800.0
266,01/02/2023,Reservation,HMJYK24N4S,01/01/2023,1,Louis Swart,Skye Ocean Nest: tiny beach home for 2 - fast ...,ZAR,772.40,27.60,0.0,800.0


In [ ]:
# Data Aggregation and Grouping
#df.groupby('group_column').sum()  # Group data and calculate sum within each group
#df.groupby(['group_column1', 'group_column2']).mean()  # Group data on multiple columns and calculate mean

In [ ]:
# Handling Categorical Variables
#pd.get_dummies(df['category_column'])  # Convert categorical variables into one-hot encoded representation

In [6]:
# Get the unique types of listings
unique_listings = df['Listing'].unique()

# Print the unique types of listings
for listing in unique_listings:
    print(listing)

# Create a dictionary to map the current listings to the new names
listing_mapping = {old_listing: f'P{i+1}' for i, old_listing in enumerate(unique_listings)}

# Use the replace() function to update the listing names
df['Listing'] = df['Listing'].replace(listing_mapping)

# Print the updated DataFrame
print(df)


Skye Sea Cottage: seaviews, deck, fireplace, wifi
Skye Beach House: seaviews, sundeck, surf, WiFi
Skye Crystal Sea Shack
Skye Seaside Retreat - Incredible sunrises/sunsets
Skye Ocean Nest: tiny beach home for 2 - fast wifi
C’ Breeze Holiday House - Beachfront Bliss!
Skye Beach Garden: Uncapped seaview, sunsets, wifi
Peaceful 2BR Beachfront Oasis
           Date                   Type Confirmation Code  Start Date  Nights  \
0    07/08/2023            Reservation        HM2RWXRDER  07/07/2023       2   
1    07/07/2023            Reservation        HMTBW5FD2J  07/06/2023       2   
2    07/07/2023  Resolution Adjustment        HM8YAZX25F  06/25/2023       3   
3    07/04/2023            Reservation        HMKD9245B4  07/03/2023       5   
4    07/04/2023            Reservation        HMCSYKHR9E  07/03/2023       3   
..          ...                    ...               ...         ...     ...   
263  01/04/2023            Reservation        HMF5ABRZAR  01/03/2023       1   
264  01/03/2

In [9]:
column_names = df.columns

# Print the column names
for column in column_names:
    print(column)

Date
Type
Confirmation Code
Start Date
Nights
Guest
Listing
Details
Reference
Currency
Amount
Paid Out
Host Fee
Cleaning Fee
Gross Earnings
Earnings Year


In [10]:
# Specify the columns to be dropped
columns_to_drop = ['Guest', 'Details', 'Reference', 'Currency', 'Amount', 'Paid Out', 'Cleaning Fee', 'Earnings Year']

# Drop the specified columns
df = df.drop(columns_to_drop, axis=1)

# Print the updated DataFrame
print(df)


           Date                   Type Confirmation Code  Start Date  Nights  \
0    07/08/2023            Reservation        HM2RWXRDER  07/07/2023       2   
1    07/07/2023            Reservation        HMTBW5FD2J  07/06/2023       2   
2    07/07/2023  Resolution Adjustment        HM8YAZX25F  06/25/2023       3   
3    07/04/2023            Reservation        HMKD9245B4  07/03/2023       5   
4    07/04/2023            Reservation        HMCSYKHR9E  07/03/2023       3   
..          ...                    ...               ...         ...     ...   
263  01/04/2023            Reservation        HMF5ABRZAR  01/03/2023       1   
264  01/03/2023            Reservation        HMP9SSCWRD  01/02/2023       2   
265  01/03/2023            Reservation        HMJMJYSH2Y  01/02/2023       1   
266  01/02/2023            Reservation        HMJYK24N4S  01/01/2023       1   
267  01/01/2023            Reservation        HMJ4HB5CE4  12/31/2022       1   

    Listing  Host Fee  Gross Earnings  

In [11]:
# Print the updated DataFrame
print(df)


           Date                   Type Confirmation Code  Start Date  Nights  \
0    07/08/2023            Reservation        HM2RWXRDER  07/07/2023       2   
1    07/07/2023            Reservation        HMTBW5FD2J  07/06/2023       2   
2    07/07/2023  Resolution Adjustment        HM8YAZX25F  06/25/2023       3   
3    07/04/2023            Reservation        HMKD9245B4  07/03/2023       5   
4    07/04/2023            Reservation        HMCSYKHR9E  07/03/2023       3   
..          ...                    ...               ...         ...     ...   
263  01/04/2023            Reservation        HMF5ABRZAR  01/03/2023       1   
264  01/03/2023            Reservation        HMP9SSCWRD  01/02/2023       2   
265  01/03/2023            Reservation        HMJMJYSH2Y  01/02/2023       1   
266  01/02/2023            Reservation        HMJYK24N4S  01/01/2023       1   
267  01/01/2023            Reservation        HMJ4HB5CE4  12/31/2022       1   

    Listing  Host Fee  Gross Earnings  

In [12]:

# Specify the desired column order
desired_columns = ['Confirmation Code','Type','Start_Date','Date', 'Nights', 'Listing', 'Host Fee','Gross Earnings']

# Reorder the columns
df = df.loc[desired_columns]

# Print the updated DataFrame
print(df)


KeyError: "None of [Index(['Confirmation Code', 'Type', 'Start_Date', 'Date', 'Nights', 'Listing',\n       'Host Fee', 'Gross Earnings'],\n      dtype='object')] are in the [index]"

### Step 2: Calculate key financial metrics


In [5]:
# Calculate financial metrics such as Net Rental Income, Occupancy Rate, ROI, etc.

In [19]:

# Calculate total earnings
total_earnings = df['Gross Earnings'].sum()

# Calculate average earnings per night
average_earnings_per_night = df['Gross Earnings'].mean()

# Calculate total nights
total_nights = df['Nights'].sum()

# Calculate occupancy rate
occupied_nights = df['Nights'].loc[df['Type'] == 'Booking'].sum()
occupancy_rate = (occupied_nights / total_nights) * 100

# Print the financial analysis results
print("Financial Analysis:")
print("Total Earnings: R{:.2f}".format(total_earnings))
print("Average Earnings per Night: R{:.2f}".format(average_earnings_per_night))
print("Total Nights: {}".format(total_nights))
print("Occupancy Rate: {:.2f}%".format(occupancy_rate))

Financial Analysis:
Total Earnings: R835046.50
Average Earnings per Night: R3139.27
Total Nights: 774
Occupancy Rate: 0.00%


In [14]:

from datetime import datetime
# Convert the 'start_date' column to datetime if it is not already in datetime format
df['Start_Date'] = pd.to_datetime(df['Start_Date'])

# Calculate the earliest start date
earliest_start_date = df['Start_Date'].min()


# Calculate the number of nights to date since the earliest start date
today = datetime.now().date()  # Get the current date
nights_to_date = (today - earliest_start_date.date()).days

# Print the number of nights to date
print("Number of nights to date since the earliest start date:", nights_to_date)

#Print the number of nights guests could book in 8 units in 189 days
print(189*5) 
774/945*100

KeyError: 'Start_Date'

In [15]:
835046.50/189 #gross earnings in the last 189 daysb

4418.23544973545

In [16]:
monthly_gross_earnings = total_earnings/6.3 # R132547.06 per month
print(monthly_gross_earnings) 

# Total monthly expenses
cleaning = 7500 
gardening = 350
maintenance = 14000
tax = 6000
municipal = 10000

# Calculate 3% of gross earnings (airbnb host fee)
three_percent_host_fee = monthly_gross_earnings * 0.03
print(three_percent_host_fee)

monthly_expenses = cleaning + gardening + maintenance + tax + municipal + three_percent_host_fee #R39516 per month  municipal
print(monthly_expenses) 

# Calculate net earnings
net_earnings = monthly_gross_earnings - monthly_expenses

# Print the net earnings
print("Net Earnings: R{:.2f}".format(net_earnings))

132547.0634920635
3976.4119047619047
41826.4119047619
Net Earnings: R90720.65


In [17]:
monthly_gross_earnings = 835046.50/6.3 # R132547.06 per month
print(monthly_gross_earnings) 

# Calculate 3% of gross earnings (airbnb host fee)
three_percent_host_fee = monthly_gross_earnings * 0.03
print(three_percent_host_fee) #3976.41

monthly_expenses = 7500 + 350 + 15000 + 6666 + 10000 + three_percent_host_fee #R39516 per month for cleaning, gardening, maintenance, provisional tax, 79 & 401 kusweg municipal
print(monthly_expenses) 

# Calculate net earnings
net_earnings_rands = monthly_gross_earnings - monthly_expenses
print(net_earnings_rands)

# Print the net earnings
print("Net Earnings: R{:.2f}".format(net_earnings))


132547.0634920635
3976.4119047619047
43492.4119047619
89054.65158730159
Net Earnings: R90720.65


In [18]:
net_earnings_rands = 89054.65  # Net earnings in South African Rand (ZAR)

# Define the exchange rate
exchange_rate = 0.041  # 1 ZAR = 0.041 GBP

# Calculate net earnings in pounds
net_earnings_pounds = net_earnings_rands * exchange_rate

# Print the result
print("Net Earnings in Pound: £{:.2f}".format(net_earnings_pounds))


Net Earnings in Pound: £3651.24


In [138]:
total_profit = 2000000  # Total profit in your local currency since inception
property = 2500000
renovation = 600000
initial_investment = property + renovation  # Initial investment in your local currency

# Calculate ROI
roi = (total_profit / initial_investment) * 100

# Print the ROI
print("ROI: {:.2f}%".format(roi))

ROI: 64.52%


In [ ]:
# Step 3: Visualization and exploratory analysis
# Visualize the financial data and metrics using matplotlib and seaborn


In [ ]:
# Step 4: Time series analysis
# Perform time series analysis, seasonal decomposition, and trend analysis using statsmodels


In [ ]:
# Step 5: Time series forecasting
# Utilize ARIMA or other forecasting techniques to predict future rental income and profitability


In [ ]:
# Step 6: Sensitivity and scenario analysis
# Conduct sensitivity analysis by modifying key financial variables and observing their impact on profitability


In [ ]:
# Step 7: Risk assessment and mitigation
# Apply statistical techniques to identify risks and uncertainties, and develop risk mitigation strategies


In [ ]:
# Step 8: Data visualization and reporting
# Create visualizations and reports using matplotlib, seaborn, or other libraries to communicate the analysis findings
